<a href="https://colab.research.google.com/github/INFINITY-RUBER/Curso_Deep_Learning_Con_TernsorFlow_Machine-Learning_e_IA-/blob/master/scripts/tema09/05-deepdream.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Dream

In [1]:
pip install tensorflow==1.14

     |████████████████████████████████| 109.2MB 38kB/s 
     |████████████████████████████████| 3.2MB 35.7MB/s 
     |████████████████████████████████| 491kB 55.4MB/s 
  Found existing installation: tensorboard 2.2.2
    Uninstalling tensorboard-2.2.2:
      Successfully uninstalled tensorboard-2.2.2
  Found existing installation: tensorflow-estimator 2.2.0
    Uninstalling tensorflow-estimator-2.2.0:
      Successfully uninstalled tensorflow-estimator-2.2.0
  Found existing installation: tensorflow 2.2.0
    Uninstalling tensorflow-2.2.0:
      Successfully uninstalled tensorflow-2.2.0


In [0]:
!cd model

In [8]:
!wget http://www.vlfeat.org/matconvnet/models/beta16/imagenet-vgg-verydeep-19.mat

--2020-06-13 00:46:50--  http://www.vlfeat.org/matconvnet/models/beta16/imagenet-vgg-verydeep-19.mat
Resolving www.vlfeat.org (www.vlfeat.org)... 64.90.48.57
Connecting to www.vlfeat.org (www.vlfeat.org)|64.90.48.57|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.vlfeat.org/matconvnet/models/beta16/imagenet-vgg-verydeep-19.mat [following]
--2020-06-13 00:46:51--  https://www.vlfeat.org/matconvnet/models/beta16/imagenet-vgg-verydeep-19.mat
Connecting to www.vlfeat.org (www.vlfeat.org)|64.90.48.57|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 576042600 (549M)
Saving to: ‘imagenet-vgg-verydeep-19.mat’

imagenet-vgg-veryde 100%[===================>] 549.36M  18.3MB/s    in 31s     

2020-06-13 00:47:23 (17.7 MB/s) - ‘imagenet-vgg-verydeep-19.mat’ saved [576042600/576042600]



In [0]:
import os
import matplotlib.pyplot as plt
import numpy as np
import PIL.Image
import tensorflow as tf
from io import BytesIO

In [0]:
graph = tf.Graph()
session = tf.InteractiveSession(graph=graph)

In [0]:
model_nn = "../../models/inception5h/tensorflow_inception_graph.pb"
with tf.gfile.FastGFile(model_nn, 'rb') as f:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())

In [0]:
t_input = tf.placeholder(np.float32, name='input')
imagenet_mean = 117.0
t_preprocessed = tf.expand_dims(t_input-imagenet_mean,0)
tf.import_graph_def(graph_def, {'input':t_preprocessed})

In [0]:
layers = [op.name for op in graph.get_operations() if op.type=="Conv2D" and "import/" in op.name]

In [0]:
layers

In [0]:
feature_num = [int(graph.get_tensor_by_name(name+':0').get_shape()[-1]) for name in layers]

In [0]:
feature_num

In [0]:
layer = "mixed4d_3x3_bottleneck_pre_relu"
channel = 139
im_noise = np.random.uniform(size = (224,224,3))+100.0

In [0]:
def show_array(a, fmt = "jpeg"):
    a = np.uint8(np.clip(a,0,1)*255)
    f = BytesIO()
    PIL.Image.fromarray(a).save(f, fmt)
    plt.imshow(a)

In [0]:
def T(layer):
    return graph.get_tensor_by_name("import/%s:0"%layer)

In [0]:
def tf_func(*argtypes):
    placeholders = list(map(tf.placeholder, argtypes))
    def wrap(f):
        out = f(*placeholders)
        def wrapper(*args, **kw):
            return out.eval(dict(zip(placeholders, args)), session=kw.get('session'))
        return wrapper
    return wrap

In [0]:
def resize(img, size):
    img = tf.expand_dims(img,0)
    return tf.image.resize_bilinear(img, size)[0,:,:,:]

In [0]:
def calc_grad_tiled(img, t_grad, tile_size=512):
    sz = tile_size
    h,w = img.shape[:2]
    sx, sy = np.random.randint(sz,size = 2)
    img_shift = np.roll(np.roll(img,sx,1),sy,0)
    grad = np.zeros_like(img)
    for y in range(0, max(h-sz//2, sz), sz):
        for x in range(0, max(w-sz//2, sz), sz):
            sub = img_shift[y:y+sz, x:x+sz]
            g = session.run(t_grad, {t_input:sub})
            grad[y:y+sz, x:x+sz] = g
    return np.roll(np.roll(grad, -sx, 1), -sy,0)

In [0]:
def render_deepdream(t_obj, img0=im_noise, iter_n = 10, step = 1.5, octave_n = 4, octave_scale = 1.4):
    t_score = tf.reduce_mean(t_obj)
    t_grad = tf.gradients(t_score, t_input)[0]
    img = img0
    octaves = []
    for i in range(octave_n-1):
        hw = img.shape[:2]
        lo = resize(img, np.int32(np.float(hw)/octave_scale))
        hi = img-resize(lo, hw)
        img = lo
        octaves.append(hi)
    
    for octave in range(octave_n):
        if octave > 0:
            hi = octaves[-octave]#empezamos desde el último octave la reconstrucción
            img = resize(img, hi.shape[:2]) + hi
        for i in range(iter_n):
            g = calc_grad_tiled(img, t_grad)
            img += g*(step / (np.abs(g).mean()+1e-7))
            print(".", end=" ")
        show_array(img/255.0)

In [0]:
resize = tf_func(np.float32, np.int32)(resize)